In [49]:
import numpy as np
import re
import sys
import os

In [50]:
def decode_n_bytes(image, n):
    word = image.read(n)
    return int.from_bytes(word, byteorder='little')

In [112]:
def get_metadata(file_name):
    image = open(file_name, 'rb')
    trash = image.read(2)
    size = decode_n_bytes(image, 4)
    trash = image.read(4)
    offset = decode_n_bytes(image, 4)
    trash = image.read(4)
    width = decode_n_bytes(image, 4)
    height = decode_n_bytes(image, 4)
    trash = image.read(2)
    bpp = decode_n_bytes(image, 2)
    print(bpp)
    bytes_per_pixel = bpp // 8
    return (size, offset, width, height, bytes_per_pixel)

In [113]:
def image_to_array_grey(file_name):
    meta = get_metadata(file_name)
    offset = meta[1]
    width = meta[2]
    height = meta[3]
    bpp = meta[4]
    image = open(file_name, 'rb')
    trash = image.read(offset)
    array = np.empty((height, width * 2), dtype = int)
    for i in range(height):
        for j in range(0, width * 2, 2):
            byte = image.read(1)
            number = int.from_bytes(byte, byteorder='big')
            array[height-1-i][j] = number
            array[height-1-i][j+1] = number
            image.read(bpp - 1)
    return array

In [125]:
def image_to_array_colour(file_name):
    meta = get_metadata(file_name)
    offset = meta[1]
    width = meta[2]
    height = meta[3]
    bpp = meta[4]
    if (width * bpp) % 4 == 0:
        fill_to_4 = 0
    else:
        fill_to_4 = 4 - (width * bpp) % 4
    print(height, width, bpp)
    image = open(file_name, 'rb')
    trash = image.read(offset)
    array = np.empty((height, width * 2), dtype = int)
    for i in range(height):
        for j in range(0, width * 2, 2):
            sum = 0
            for k in range(bpp):
                byte = image.read(1)
                sum = sum + int.from_bytes(byte, byteorder='big')            
            mean = sum / bpp
            array[height-1-i][j] = mean
            array[height-1-i][j+1] = mean
        if fill_to_4 > 0:
            trash = image.read(fill_to_4)
    return array

In [115]:
#function to convert number to char
def num2char(numArray):
    [r1, r2, r3, r4, r5, r6, r7] = [np.array([range(0,37)]), np.array([range(37,74)]), np.array([range(74,111)]),
                            np.array([range(111,147)]), np.array([range(147,184)]), np.array([range(184,220)]),
                            np.array([range(220,256)])]
    charArray = ['.', ',', '-', ':', '+', '$', '#']
    fullArrayStrNeg = ""
    fullArrayStrPos = ""
    for i in numArray:
        for j in i:
            if j in r1:
                fullArrayStrNeg += charArray[0]
                fullArrayStrPos += charArray[6]
            elif j in r2:
                fullArrayStrNeg += charArray[1]
                fullArrayStrPos += charArray[5]
            elif j in r3:
                fullArrayStrNeg += charArray[2]
                fullArrayStrPos += charArray[4]
            elif j in r4:
                fullArrayStrNeg += charArray[3]
                fullArrayStrPos += charArray[3]
            elif j in r5:
                fullArrayStrNeg += charArray[4]
                fullArrayStrPos += charArray[2]
            elif j in r6:
                fullArrayStrNeg += charArray[5]
                fullArrayStrPos += charArray[1]
            else:
                fullArrayStrNeg += charArray[6]
                fullArrayStrPos += charArray[0]
        fullArrayStrNeg += '\n'
        fullArrayStrPos += '\n'
    return fullArrayStrNeg, fullArrayStrPos

In [116]:
def write2file(charArrayNeg, charArrayPos):
    finalFileNeg = open("finalFileNeg.txt","w")
    finalFileNeg.write(charArrayNeg)
    finalFilePos = open("finalFilePos.txt","w")
    finalFilePos.write(charArrayPos)

In [127]:
#getting argument (numpy array)
hexArray = image_to_array_colour("ludzie.bmp")
charArrNeg, charArrPos = num2char(hexArray)
write2file(charArrNeg, charArrPos)

24
627 1200 3
